In [120]:
import pandas as pd
import re

cases_all = []
for y in [2018, 2019, 2020, 2021, 2022]:
    
    # print("year:", y)
    df = pd.read_csv(f"{y}-exportList.csv", sep=';')
    cases = df.filter(["Postcode", "Leges uitspraak", "Huurprijs voor uitspraak", "Huurprijs na uitspraak", "Tijdelijk verlaagde huurprijs", "Verzoeker"], axis=1)
    column_names = ["Zip", "Winner", "RentBefore", "RentAfter", "RentLowered", "Complainer"]
    cases = cases.set_axis(column_names, axis=1)
    # cases["Winner"] = cases["Winner"].astype("string")
    cases["Winner"] = cases.apply(lambda row: "Complainer" if re.search(r".*verzoeker.*",str(row.Winner)) else "Defendant", axis=1)
    cases["Year"] = y
    # print(cases.head(2))
    cases_all.append(cases)
print(len(cases_all))
cases_all = pd.concat(cases_all, ignore_index=True)
cases_all.to_csv("huurcommissie_cases.csv", index=False)
cases_all.head(2)

5


,Zip,Winner,RentBefore,RentAfter,RentLowered,Complainer,Year
0,9712CW,Complainer,365,"109,5",NaN,Huurder,2018
1,2034RV,Complainer,NaN,NaN,NaN,Huurder,2018


In [121]:
# Huurcommissie public cases data https://www.huurcommissie.nl/over-de-huurcommissie/uitspraken/export-openbaar-register
df = pd.read_csv("huurcommissie_cases.csv", index_col=None)
# df = df.applymap(str.strip)
# df["Zip"] = df["Zip"].astype(pd.StringDtype())
df["End"] = df["Zip"].str[4:].astype("string")
df["Code"] = df["Zip"].str[:4]
df["Code"] = pd.to_numeric(df["Code"], errors="coerce")
df = df.dropna(subset=["Code"])
df["Code"] = df["Code"].astype(int)

df.dtypes

Zip            object
Winner         object
RentBefore     object
RentAfter      object
RentLowered    object
Complainer     object
Year            int64
End            string
Code            int32
dtype: object

In [122]:
#3000 AA - 3099 ZZ: Rotterdam (https://en.youbianku.com/netherlands)

df_rot = df.pipe(lambda x: x.loc[(x['Code'] <= 3099) & (x["Code"] >= 3000)])
# df_rot = df[int(df.Zip.str[:3]) == 777]
df_rot


,Zip,Winner,RentBefore,RentAfter,RentLowered,Complainer,Year,End,Code
17,3014EB,Complainer,NaN,NaN,NaN,Huurder,2018,EB,3014
18,3051LK,Complainer,"732,94","694,23",NaN,Huurder,2018,LK,3051
19,3081BX,Defendant,"456,19","456,19",NaN,Huurder,2018,BX,3081
24,3085GT,Complainer,"544,98","544,98",NaN,Huurder,2018,GT,3085
27,3022GB,Complainer,"569,74","455,79",NaN,Huurder,2018,GB,3022
...,...,...,...,...,...,...,...,...,...
8342,3071SB,Defendant,"703,68",NaN,NaN,Huurder,2022,SB,3071
8346,3082JC,Complainer,"710,00","527,30",NaN,Huurder,2022,JC,3082
8347,3082JC,Complainer,"710,00",NaN,"426,00",Huurder,2022,JC,3082
8349,3082LN,Defendant,NaN,NaN,NaN,Huurder,2022,LN,3082


In [123]:

#  Not easy to find, but physical location for all postal codes in the Netherlands from http://www.geonames.org/export/
physical_locations = pd.read_csv("NL_full.txt", header=None, delimiter=r"\s+", on_bad_lines="skip")
physical_locations.columns
d = physical_locations.iloc[:, [1, 2, 8, 9]]
column_names = ["Code", "End", "Lat", "Lon"]
d = d.set_axis(column_names, axis=1)
d.head(5)
d_rot = d.pipe(lambda x: x.loc[(x['Code'] <= 3099) & (x["Code"] >= 3000)])
d_rot.head(5)

,Code,End,Lat,Lon
323824,3011,AA,51.9203,4.4809
323825,3011,AB,51.9193,4.4815
323826,3011,AC,51.9195,4.4816
323827,3011,AD,51.9230,4.4791
323828,3011,AE,51.9203,4.4804


In [124]:
# complaints = pd.merge(df_rot, d_rot, on=["Code", "End"], how="inner")
complaints_all = pd.merge(df , d, on=["Code", "End"], how="inner")
# complaints.head(5)
# complaints.describe()
complaints_all.to_csv("complaints_all.csv", index=False)
print(complaints_all.head(1))
complaints_all.to_json("complaints_all.json", orient="records")

      Zip      Winner RentBefore RentAfter RentLowered Complainer  Year End  \
0  9712CW  Complainer        365     109,5         NaN    Huurder  2018  CW   

   Code      Lat     Lon  
0  9712  53.2194  6.5601  
